In [59]:
import pandas as pd
df=pd.read_csv('used_car_dataset.csv')

In [60]:
df2=pd.read_csv('used_car_dataset.csv')

In [3]:
import numpy as np

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error,r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pickle

In [69]:
df

,car_name,car_price_in_rupees,kms_driven,fuel_type,city,year_of_manufacture
0,1,₹ 4.45 Lakh,"22,402 km",1,1,2016
1,2,₹ 2.93 Lakh,"10,344 km",1,2,2019
2,3,₹ 22.49 Lakh,"12,999 km",2,3,2021
3,4,₹ 6.95 Lakh,"45,000 km",1,4,2016
4,5,₹ 12 Lakh,"11,193 km",1,2,2019
...,...,...,...,...,...,...
2100,945,₹ 3.6 Lakh,"42,158 km",2,2,2015
2101,352,₹ 22 Lakh,"68,862 km",2,8,2013
2102,103,₹ 8.38 Lakh,"37,622 km",1,5,2018
2103,946,₹ 6.75 Lakh,"64,726 km",1,1,2017


In [70]:
import re

df = pd.read_csv("used_car_dataset.csv")

def convert_price_to_numeric(price_str):
    match = re.match(r'₹ (\d+(?:\.\d+)?) (Lakh|Crore)?', price_str)
    
    if match:
        value, unit = match.groups()
        value = float(value)
        
        # Multiply by 100,000 for Lakh and 10,000,000 for Crore
        multiplier = 100000 if unit == 'Lakh' else 10000000
        return int(value * multiplier)
    else:
        # Handle unexpected formats
        return None

# Apply the function to the 'car_price_in_rupees' column
df['car_price_in_rupees'] = df['car_price_in_rupees'].apply(convert_price_to_numeric)

# Save the updated DataFrame to a new CSV file
#df.to_csv("used_car_price_updated.csv", index=False)


In [71]:
df

,car_name,car_price_in_rupees,kms_driven,fuel_type,city,year_of_manufacture
0,Hyundai Grand i10 Magna 1.2 Kappa VTVT [2017-2...,445000.0,"22,402 km",Petrol,Mumbai,2016
1,Maruti Suzuki Alto 800 Lxi,293000.0,"10,344 km",Petrol,Kolkata,2019
2,Tata Safari XZ Plus New,2249000.0,"12,999 km",Diesel,Bangalore,2021
3,Maruti Suzuki Ciaz ZXI+,695000.0,"45,000 km",Petrol,Thane,2016
4,Jeep Compass Sport Plus 1.4 Petrol [2019-2020],1200000.0,"11,193 km",Petrol,Kolkata,2019
...,...,...,...,...,...,...
2100,Ford Figo Titanium1.5 TDCi,360000.0,"42,158 km",Diesel,Kolkata,2015
2101,MINI Cooper Countryman Cooper D,2200000.0,"68,862 km",Diesel,Hyderabad,2013
2102,Hyundai Verna 1.6 VTVT SX,838000.0,"37,622 km",Petrol,Chennai,2018
2103,Maruti Suzuki Ciaz VXi+ AT,675000.0,"64,726 km",Petrol,Mumbai,2017


In [72]:
def convert_kms_to_numeric(kms_str):
    try:
        # Extract numerical value and remove commas
        value = int(kms_str.replace(',', '').split()[0])
        return value
    except ValueError:
        # Handle unexpected formats or non-numeric values
        return None

# Apply the function to the 'kms_driven' column
df['kms_driven'] = df['kms_driven'].apply(convert_kms_to_numeric)

# Save the updated DataFrame to a new CSV file
#df.to_csv("used_car_dataset_updated.csv", index=False)

In [13]:
df

,car_name,car_price_in_rupees,kms_driven,fuel_type,city,year_of_manufacture
0,Hyundai Grand i10 Magna 1.2 Kappa VTVT [2017-2...,445000.0,22402,Petrol,Mumbai,2016
1,Maruti Suzuki Alto 800 Lxi,293000.0,10344,Petrol,Kolkata,2019
2,Tata Safari XZ Plus New,2249000.0,12999,Diesel,Bangalore,2021
3,Maruti Suzuki Ciaz ZXI+,695000.0,45000,Petrol,Thane,2016
4,Jeep Compass Sport Plus 1.4 Petrol [2019-2020],1200000.0,11193,Petrol,Kolkata,2019
...,...,...,...,...,...,...
2100,Ford Figo Titanium1.5 TDCi,360000.0,42158,Diesel,Kolkata,2015
2101,MINI Cooper Countryman Cooper D,2200000.0,68862,Diesel,Hyderabad,2013
2102,Hyundai Verna 1.6 VTVT SX,838000.0,37622,Petrol,Chennai,2018
2103,Maruti Suzuki Ciaz VXi+ AT,675000.0,64726,Petrol,Mumbai,2017


In [73]:
num_col=['car_price_in_rupees','kms_driven','year_of_manufacture']
cat_cols=['car_name','fuel_type','city']

In [74]:
temp=[]
for i in cat_cols:
   temp.append(df[i].unique().flatten()) 

In [75]:
temp

[array(['Hyundai Grand i10 Magna 1.2 Kappa VTVT [2017-2020]',
        'Maruti Suzuki Alto 800 Lxi', 'Tata Safari XZ Plus New',
        'Maruti Suzuki Ciaz ZXI+',
        'Jeep Compass Sport Plus 1.4 Petrol [2019-2020]',
        'Maruti Suzuki Eeco 5 STR WITH A/C+HTR [2019-2020]',
        'Hyundai Creta SX Plus 1.6 AT CRDI', 'Maruti Suzuki Ertiga VXi AT',
        'Toyota Etios Liva G', 'Renault Kwid 1.0 RXT AMT Opt [2016-2019]',
        'Maruti Suzuki Ciaz VDi+ SHVS', 'Maruti Suzuki Alto 800 LXi (O)',
        'Chevrolet Optra Magnum LS 2.0 TCDi',
        'Maruti Suzuki Vitara Brezza VDi',
        'Jeep Compass Longitude 2.0 Diesel [2017-2020]',
        'Toyota Corolla Altis VL AT Petrol',
        'Hyundai Elite i20 Magna Executive 1.2',
        'Mercedes-Benz EQC 400 4MATIC',
        'Ford EcoSport Titanium 1.5L Ti-VCT',
        'Hyundai i10 Magna 1.2 Kappa2',
        'Hyundai Creta 1.6 SX Plus Petrol Special Edition',
        'Porsche 718 Boxster', 'Kia Seltos HTE 1.5 Diesel',
        

In [76]:
df[cat_cols]=df[cat_cols].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105 entries, 0 to 2104
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   car_name             2105 non-null   category
 1   car_price_in_rupees  2103 non-null   float64 
 2   kms_driven           2105 non-null   int64   
 3   fuel_type            2105 non-null   category
 4   city                 2105 non-null   category
 5   year_of_manufacture  2105 non-null   int64   
dtypes: category(3), float64(1), int64(2)
memory usage: 98.4 KB


In [77]:
columnsValue={}
for col in cat_cols:
    column_labels=list(range(1,len(df[col].unique())+1))
    rep={}
    for i,j in zip(column_labels,df[col].unique()):
        rep[j]=i
    columnsValue[col]=list(df[col].unique())
    df[col].replace(rep,inplace=True)

In [78]:
df.head()

,car_name,car_price_in_rupees,kms_driven,fuel_type,city,year_of_manufacture
0,1,445000.0,22402,1,1,2016
1,2,293000.0,10344,1,2,2019
2,3,2249000.0,12999,2,3,2021
3,4,695000.0,45000,1,4,2016
4,5,1200000.0,11193,1,2,2019


In [80]:
norm = StandardScaler()

norm.fit(df[['year_of_manufacture','kms_driven']])
standardvalues=norm.transform(df[['year_of_manufacture','kms_driven']])

df['year_of_manufacture']=standardvalues[:,:1].flatten()
df['kms_driven']=standardvalues[:,1:].flatten()

pickle.dump(norm, open('StandardScaler.sav', 'wb'))

df['car_price_in_rupees'] = np.log(df['car_price_in_rupees'])
#scaling target variable
q1,q3=(df['car_price_in_rupees'].quantile([0.25,0.75]))
o1=q1-1.5*(q3-q1)
o2=q3+1.5*(q3-q1)
df=df[(df.car_price_in_rupees>=o1) & (df.car_price_in_rupees<=o2)]

In [81]:
X = df.iloc[:, [col for col in range(df.shape[1]) if col != 1]]
y = df['car_price_in_rupees']

xg_reg2 = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.4, max_depth=24, alpha=5, n_estimators=200)

xg_reg2.fit(X.values, y.values)

pickle.dump(xg_reg2, open('XGBoostDeploy.sav', 'wb'))

In [82]:
standardscaler= pickle.load(open('StandardScaler.sav', 'rb'))
mymodel = pickle.load(open('XGBoostDeploy.sav', 'rb'))

In [83]:
df.sample(2)

,car_name,car_price_in_rupees,kms_driven,fuel_type,city,year_of_manufacture
1846,193,14.905755,-0.297039,2,4,1.411657
698,466,13.521139,-0.027220,1,1,0.346844


In [84]:
df2.iloc[-1]

car_name               Hyundai Verna 1.6 VTVT SX
car_price_in_rupees                  ₹ 8.76 Lakh
kms_driven                             29,150 km
fuel_type                                 Petrol
city                                        Pune
year_of_manufacture                         2017
Name: 2104, dtype: object

In [85]:
year_odometer=pd.DataFrame(data=[[2020.0,6052]],columns=['year_of_manufacture','kms_driven'])
x=standardscaler.transform(year_odometer[['year_of_manufacture','kms_driven']]).flatten()

In [86]:
#get col uncoded value
df2.year_of_manufacture.unique()

array([2016, 2019, 2021, 2017, 2013, 2018, 2007, 2022, 2014, 2020, 2015,
       2009, 2011, 2010, 2012, 2008, 2006, 2004, 2005], dtype=int64)

In [87]:
test=pd.DataFrame(data=[[5,x[0],1,2,x[1]]],columns=X.columns)
test

,car_name,kms_driven,fuel_type,city,year_of_manufacture
0,5,1.056719,1,2,-0.504767


In [ ]:
car_name	kms_driven	fuel_type	city	year_of_manufacture

In [88]:
t=[8,x[0],1,1,x[1]]
t

[8, 1.0567193102399113, 1, 1, -0.5047672530981279]

In [89]:
pred=mymodel.predict(test)
price=np.exp(pred[0])
print(price)


501895.56


In [ ]:
# The Price is in indian rupees